In [ ]:
import pickle
import numpy as np
from src.groupedpaneldatamodels import GroupedFixedEffects, GroupedInteractiveFixedEffects
from joblib import Parallel, delayed
from tqdm import tqdm

# Bonhomme Manresa (DGP 1)

In [ ]:
def bm_run(run, G, i):
    x, y, f, g = run
    x = np.float32(x)
    y = np.atleast_3d(np.float32(y))
    f = np.float32(f)
    gfe_iterations = 10 if G < 4 else 100
    n_boot = 200 if G < 4 else 100
    m = GroupedFixedEffects(y, x, G, True, hide_progressbar=True).fit(
        max_iter=100, gfe_iterations=gfe_iterations, n_boot=n_boot
    )

    return m.to_dict()

In [ ]:
for n in (100, 200):
    for t in (20, 50):
        for G in (3, 6):
            for k in (3,):
                print(f"\n\nRunning DGP1 with n={n}, t={t}, G={G}, k={k}")
                runs = pickle.load(open(f"development/generated_data/dgp1_n{n}_t{t}_G{G}_k{k}.pkl", "rb"))

                # Wrap the iterable in tqdm to get a progress bar
                estimates = Parallel(n_jobs=-1, prefer="processes")(
                    delayed(bm_run)(run, G, i)
                    for i, run in tqdm(enumerate(runs[:200]), total=200, desc=f"DGP1 n={n}, t={t}, G={G}, k={k}")
                )

                with open(f"estimates/dgp1_n{n}_t{t}_G{G}_k{k}.pkl", "wb") as f:
                    pickle.dump(estimates, f)

## DGP 2

In [ ]:
def ab_run(run, G, i):
    x, y, f, g, *_ = run
    x = np.float32(x)
    y = np.atleast_3d(np.float32(y))
    f = np.float32(f)
    gife_iterations = 10 if G < 4 else 100
    n_boot = 200 if G < 4 else 100
    m = GroupedInteractiveFixedEffects(y, x, G, True, hide_progressbar=True).fit(max_iter=100, gife_iterations=gife_iterations, n_boot=n_boot)

    return m.to_dict()

In [ ]:
for n in (100, 200):
    for t in (20, 50):
        for G in (3, 6):
            for k in (3,):
                print(f"\n\nRunning DGP2 (ab) with n={n}, t={t}, G={G}, k={k}")
                runs = pickle.load(open(f"development/generated_data/dgp2_n{n}_t{t}_G{G}_k{k}.pkl", "rb"))

                estimates = Parallel(n_jobs=-1, prefer="processes")(
                    delayed(ab_run)(run, G, i)
                    for i, run in tqdm(enumerate(runs[:200]), total=200, desc=f"DGP2 n={n}, t={t}, G={G}, k={k}")
                )

                with open(f"estimates/dgp2_n{n}_t{t}_G{G}_k{k}.pkl", "wb") as f:
                    pickle.dump(estimates, f)

In [2]:
def sj_run(run, G, i):
    x, y, f, g, *_ = run
    x = np.float32(x)
    y = np.atleast_3d(np.float32(y))
    f = np.float32(f)
    n_boot = 100 if G < 4 else 50
    kappa = 0.005 if G < 4 else 0.00005
    m = GroupedInteractiveFixedEffects(y, x, G, True, "su_ju", R=G, hide_progressbar=True).fit(
        n_boot=n_boot, only_bfgs=True, max_iter=100, kappa=kappa, tol=1e-2
    )

    return m.to_dict()

In [3]:
for n in (100, 200):
    for t in (20, 50):
        for G in (3, 6):
            for k in (3,):
                print(f"\n\nRunning DGP2 (su_ju) with n={n}, t={t}, G={G}, k={k}")
                runs = pickle.load(open(f"development/generated_data/dgp2_n{n}_t{t}_G{G}_k{k}.pkl", "rb"))

                estimates = Parallel(n_jobs=-1, prefer="processes")(
                    delayed(sj_run)(run, G, i)
                    for i, run in enumerate(tqdm(runs[:200], desc=f"DGP2 n={n}, t={t}, G={G}, k={k}"))
                )

                with open(f"estimates/dgp2_n{n}_t{t}_G{G}_k{k}_su_ju.pkl", "wb") as f:
                    pickle.dump(estimates, f)



Running DGP2 (su_ju) with n=100, t=20, G=3, k=3


DGP2 n=100, t=20, G=3, k=3:  10%|█         | 20/200 [00:57<08:40,  2.89s/it]

KeyboardInterrupt: 

# DGP 3 (SSJ)

In [ ]:
def ssp_run(run, G, i):
    x, y, g = run
    x = np.float32(x)
    y = np.atleast_3d(np.float32(y))
    n_boot = 100 if G < 4 else 50
    kappa = 0.005 if G < 4 else 0.00005
    m = GroupedFixedEffects(y, x, G, True, "su_shi_phillips", hide_progressbar=False).fit(
        n_boot=n_boot, only_bfgs=False, max_iter=100, kappa=kappa, tol=1e-5
    )

    return m.to_dict()

Bootstrap GroupedFixedEffects@0x154ec0f50: 100%|██████████| 100/100 [00:24<00:00,  4.14it/s]


{'name': 'GroupedFixedEffects',
 'id': '0x154ec0f50',
 'fit_datetime': '2025-06-22 21:57:37',
 'fit_duration': 200.607197,
 'model_type': 'su_shi_phillips',
 'params': {'beta': array([[0.9752052 , 1.02096804, 1.03397083],
         [1.94705547, 2.10195583, 2.01913477],
         [3.03720153, 3.00897187, 2.97261922]]),
  'b': array([[0.97577067, 2.00834705, 3.05843823, 3.07416981, 3.07163485,
          1.02634178, 2.04942254, 1.94712974, 1.251697  , 3.35244533,
          1.90603913, 2.16426509, 2.33440084, 1.97608125, 1.83640093,
          2.66830622, 2.65481544, 3.53920817, 1.65807487, 0.84692417,
          0.77637368, 2.2085375 , 3.18421298, 1.47803444, 3.4898657 ,
          0.69795958, 1.48577808, 1.18225311, 0.91067578, 3.03569154,
          1.90386376, 2.57891521, 1.61671921, 3.33707301, 0.82389489,
          1.80320238, 3.47312184, 3.26002438, 2.92784255, 1.96398719,
          3.02205402, 1.34522279, 2.22040859, 0.76957915, 1.17250083,
          0.88303647, 3.20953714, 1.84311801, 0

In [5]:
for n in (100, 200):
    for t in (20, 50):
        for G in (3, 6):
            for k in (3,):
                print(f"\n\nRunning DGP3 (su_shi_phillips) with n={n}, t={t}, G={G}, k={k}")
                runs = pickle.load(open(f"development/generated_data/dgp3_n{n}_t{t}_G{G}_k{k}.pkl", "rb"))
                estimates = []
                estimates = Parallel(n_jobs=-1, prefer="processes")(
                    delayed(ssp_run)(run, G, i) for i, run in enumerate(runs[:200])
                )
                with open(f"estimates/dgp3_n{n}_t{t}_G{G}_k{k}.pkl", "wb") as f:
                    pickle.dump(estimates, f)



Running DGP3 (su_shi_phillips) with n=100, t=20, G=3, k=3


Exception ignored on calling ctypes callback function <function ExecutionEngine._raw_object_cache_notify at 0x103d58e00>:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/llvmlite/binding/executionengine.py", line 178, in _raw_object_cache_notify
    def _raw_object_cache_notify(self, data):
KeyboardInterrupt: 
Exception ignored on calling ctypes callback function <function ExecutionEngine._raw_object_cache_notify at 0x105958e00>:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/llvmlite/binding/executionengine.py", line 178, in _raw_object_cache_notify
    def _raw_object_cache_notify(self, data):
KeyboardInterrupt: 


KeyboardInterrupt: 